In [1]:
import ast
import astpretty
import sys

sys.path.insert(0, '../preprocess/')
sys.path.insert(0, '../../coarse2fine.git/src')

from sketch_generation import Sketch
from tree import SketchRepresentation

In [4]:
SKP_WORD = '<sk>'
RIG_WORD = '<]>'
LFT_WORD = '<[>'

def is_code_eq(tokens1, tokens2, not_layout=False):

    if isinstance(tokens1, SketchRepresentation):
        tokens1 = str(tokens1)
    else:
        tokens1 = ' '.join(tokens1)

    if isinstance(tokens2, SketchRepresentation):
        tokens2 = str(tokens2)
    else:
        tokens2 = ' '.join(tokens2)

    tokens1 = ['\"' if it in (RIG_WORD, LFT_WORD) else it for it in tokens1.split(' ')]
    tokens2 = ['\"' if it in (RIG_WORD, LFT_WORD) else it for it in tokens2.split(' ')]

    if len(tokens1) != len(tokens2):
        return False

    return all(map(lambda tk1, tk2: tk1 == tk2, tokens1, tokens2))

In [14]:
code1 = 'x = b + a'
code2 = 'x = a + b'

astpretty.pprint(ast.parse(code1).body[0], indent=' ' * 4)
astpretty.pprint(ast.parse(code2).body[0], indent=' ' * 4)

Assign(
    lineno=1,
    col_offset=0,
    targets=[Name(lineno=1, col_offset=0, id='x', ctx=Store())],
    value=BinOp(
        lineno=1,
        col_offset=4,
        left=Name(lineno=1, col_offset=4, id='b', ctx=Load()),
        op=Add(),
        right=Name(lineno=1, col_offset=8, id='a', ctx=Load()),
    ),
)
Assign(
    lineno=1,
    col_offset=0,
    targets=[Name(lineno=1, col_offset=0, id='x', ctx=Store())],
    value=BinOp(
        lineno=1,
        col_offset=4,
        left=Name(lineno=1, col_offset=4, id='a', ctx=Load()),
        op=Add(),
        right=Name(lineno=1, col_offset=8, id='b', ctx=Load()),
    ),
)
